<br>
<br>
<p align="center">
  <img src="images/SHAP-vitruvian.png" 
       width="50">
</p>
<p align="center">
  <img src="images/qsffe.png" 
       width="400">
</p>
</p>
<p align="center">
  <img src="images/tuv.png" 
       width="150">
</p>

# Exact Diagonalization Basics

## Importing modules

This notebook will follow very closely the documentation found [here](https://scipost.org/SciPostPhys.2.1.003/pdf). The first step is to import the necessary modules. Besides `numPy`, we will need:
- The basis module `quspin.basis` to build our Hilbert space
- The operators module `quspin.operators` to build our Hamiltonian and operators

In [1]:
from quspin.operators import hamiltonian
from quspin.basis import spin_basis_1d 
import numpy as np

## Our Hamiltonian

Let us start with the XXZ Hamiltonian in an external field $h_z$

$$ H = \sum\limits_{j = 0}^{L - 2} {\frac{{{J_{xy}}}}{2}(S_{j + 1}^ + S_j^ -  + {\rm{h.c.}})}  + {J_{zz}}S_{j + 1}^zS_j^z + {h_z}\sum\limits_{j = 0}^{L - 1} {S_j^z}, $$

and choose our parameters:
- The lattice size $L=12$
- The couplings $J_{xy}=\sqrt 2$ and $J_{zz}=1$
- The external field $h_z=1/{\sqrt 3}$

In [2]:
L, Jxy, Jzz, hz = 12, np.sqrt(2.0), 1.0, 1.0/np.sqrt(3.0)
print(' The system size, xy coupling, zz coupling and external fields are:\n','L =',L,
      '\n Jxy = ',round(Jxy,2),'\n Jzz = ',Jzz,'\n h_z = ',round(hz,2))

 The system size, xy coupling, zz coupling and external fields are:
 L = 12 
 Jxy =  1.41 
 Jzz =  1.0 
 h_z =  0.58


## Basis

A few observations:
- The optional argument `pauli = False` prevents the code to use the Pauli matrices instead of spin matrices as needed. 
- We choose half-filling and the number of spins-up is rounded down. 
- Furthermore, since this Hamiltonian conserves parity (which in this case corresponds to reflection with respect to the middle of the chain), the Hilbert space is divided into two subspaces and we must choose one of them. To choose positive parity we set the optional argument `pblock=1`.

In [3]:
basis = spin_basis_1d(L, pauli=False, Nup = L//2, pblock = 1)
print(' The basis vectors are given by:\n\n',basis)

 The basis vectors are given by:

 reference states: 
       0.  |1 1 1 1 1 1 0 0 0 0 0 0>
       1.  |1 1 1 1 1 0 1 0 0 0 0 0>
       2.  |1 1 1 1 1 0 0 1 0 0 0 0>
       3.  |1 1 1 1 1 0 0 0 1 0 0 0>
       4.  |1 1 1 1 1 0 0 0 0 1 0 0>
       5.  |1 1 1 1 1 0 0 0 0 0 1 0>
       6.  |1 1 1 1 1 0 0 0 0 0 0 1>
       7.  |1 1 1 1 0 1 1 0 0 0 0 0>
       8.  |1 1 1 1 0 1 0 1 0 0 0 0>
       9.  |1 1 1 1 0 1 0 0 1 0 0 0>
      10.  |1 1 1 1 0 1 0 0 0 1 0 0>
      11.  |1 1 1 1 0 1 0 0 0 0 1 0>
      12.  |1 1 1 1 0 1 0 0 0 0 0 1>
      13.  |1 1 1 1 0 0 1 1 0 0 0 0>
      14.  |1 1 1 1 0 0 1 0 1 0 0 0>
      15.  |1 1 1 1 0 0 1 0 0 1 0 0>
      16.  |1 1 1 1 0 0 1 0 0 0 1 0>
      17.  |1 1 1 1 0 0 1 0 0 0 0 1>
      18.  |1 1 1 1 0 0 0 1 1 0 0 0>
      19.  |1 1 1 1 0 0 0 1 0 1 0 0>
      20.  |1 1 1 1 0 0 0 1 0 0 1 0>
      21.  |1 1 1 1 0 0 0 1 0 0 0 1>
      22.  |1 1 1 1 0 0 0 0 1 1 0 0>
      23.  |1 1 1 1 0 0 0 0 1 0 1 0>
      24.  |1 1 1 1 0 0 0 0 1 0 0 1>
                     

## Operators 

We will choose open boundary conditions in this case. To define the operators we use [list comprehensions](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions). 

In [4]:
def spin_operators(coupling, L):
    return [[coupling,i,i+1] for i in range(L-1)]

In [5]:
J_zz = spin_operators(Jzz, L)
J_xy = spin_operators(Jxy/2.0, L)
h_z=[[hz,i] for i in range(L)]

## Hamiltonian

In this case we have a static Hamiltonian. The corresponding syntax is:

In [6]:
static, dynamic = [["+-",J_xy],["-+",J_xy],["zz",J_zz],["z",h_z]], []
H_XXZ = hamiltonian(static,dynamic,basis=basis,dtype=np.float64)

Hermiticity check passed!
Symmetry checks passed!
Particle conservation check passed!


The eigenvalues are obtained using:

In [7]:
E = H_XXZ.eigvalsh()

print('Some eigenvalues are:\n')
print('Lowest:',[round(el,1) for el in sorted(E)[0:5]],'\n')
print('Highest:',[round(el,1) for el in sorted(E)[len(E)-5:]])

Some eigenvalues are:

Lowest: [-6.6, -5.7, -5.7, -5.0, -5.0] 

Highest: [3.5, 3.5, 3.8, 3.8, 4.1]
